## Identifying Poker Hands

As it turns out, identifying poker hands is a great way to learn how sequence and set rules work. Begin by importing the standard libraries and creating a new rules engine.

## ♥ ♦ ♠ ♣

In [1]:
import os, sys
sys.path.insert(1, os.path.abspath('..\\..\\..'))
from thoughts.rules_engine import RulesEngine
import pprint

engine = RulesEngine()

## High Card

Detecting high cards is relatively straight forward. Since any card could be a high card* we'll simply forward the card's rank information on as a high card.

Don't worry for now about picking the highest scoring one of these results. These initial rules are simply to generate candidate card rankings for all possibilities. Later we'll pick the highest scoring pattern out of the candidates.

*Except for 2's, since there would always be guaranteed at least at 3 in 10 cards shared among 2 people using 2 decks, but will ignore that anomaly.

In [2]:
# define the rule
high_card_rule = {"#when": [{"suit": "?suit", "rank": "?rank1"}],
                  "#then": {"high-card": "?rank1", "score": 1}}
engine.add_rule(high_card_rule)

# assert a hand to test the rule
hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "5", "suit": "clubs"},
    {"rank": "7", "suit": "hearts"},
    {"rank": "9", "suit": "diamonds"},
    {"rank": "K", "suit": "clubs"}]
result = engine.process(hand)
pprint.pprint(result)

[{'high-card': '3', 'score': 1},
 {'high-card': '5', 'score': 1},
 {'high-card': '7', 'score': 1},
 {'high-card': '9', 'score': 1},
 {'high-card': 'k', 'score': 1}]


## One Pair

To detect pairs, we are interested whenever there are two cards that follow in sequence that have the same rank.

In the rule below, note that the ?rank1 variable is the same between both constituents in the #when sequence. The engine will use this informtion to make sure the value of the rank in the second card matches the rank in the first card.

Also note the #seq-type of "allow-junk" as part of the rule. This allows for extra constituents to be in-between the constituents we are looking for. Without this, the second card in the pair would have to come immediately after first card in the sequence. For example, it would detect 2♥ followed by 2♦, but not 2♥ followed by 3♠ followed by 2♦. The 3♠ is the "junk" consituent which is ignored whenever #seq-type is "allow-junk".

In [3]:
# define the rule
one_pair = {"#when": [{"rank": "?rank1"}, {"rank": "?rank1"}],
            "#seq-type": "allow-junk",
            "#then": {"one-pair": "?rank1", "score": 2}}
engine.add_rule(one_pair)

# assert a hand to test the rule
hand = [
    {"rank": "J", "suit": "spades"}, 
    {"rank": "3", "suit": "clubs"},
    {"rank": "J", "suit": "hearts"},
    {"rank": "7", "suit": "diamonds"},
    {"rank": "5", "suit": "clubs"}]
result = engine.process(hand)
pprint.pprint(result, sort_dicts=False)

[{'high-card': 'j', 'score': 1},
 {'high-card': '3', 'score': 1},
 {'one-pair': 'j', 'score': 2},
 {'high-card': 'j', 'score': 1},
 {'high-card': '7', 'score': 1},
 {'high-card': '5', 'score': 1}]


## Two Pair

Detecting two-pair works by matching whenever you find a single (one) pair, followed by another single (one) pair, with junk cards allowed in between.

Here the ranks can be the same, but do not have to be the same. This means this rule will detect four 4's as two pairs, though we know it also be better known as four of a kind. That's OK - we'll let the engine detect the two pairs and ALSO detect the four of a kind. Technically, that's an accurate identification the allowed patterns. In general it's better for the engine to over-generate matches as possible "ideas", and then let another set of rules filter these down into the correct conclusion based on some other criteria.

In [4]:
# add the rule
rule = {"#when": [{"one-pair": "?rank1"}, {"one-pair": "?rank2"}],
    "#seq-type": "allow-junk",
     "#then": {"two-pair": {"pair1": "?rank1", "pair2": "?rank2"}, "score": 3}}
engine.add_rule(rule)

# test it
hand = [
    {"rank": "J", "suit": "spades"}, 
    {"rank": "J", "suit": "spades"},
    {"rank": "7", "suit": "hearts"},
    {"rank": "3", "suit": "clubs"},
    {"rank": "7", "suit": "hearts"}
]
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': 'j', 'score': 1},
 {'one-pair': 'j', 'score': 2},
 {'high-card': 'j', 'score': 1},
 {'high-card': '7', 'score': 1},
 {'high-card': '3', 'score': 1},
 {'two-pair': {'pair1': 'j', 'pair2': '7'}, 'score': 3},
 {'high-card': '7', 'score': 1}]


## Three of a Kind

Three of a Kind works similiar to the one-pair and two-pair rules. Look for a sequence of three cards with the same rank, and allow junk cards in between.

In [5]:
# add the rule
rule = {"#when": [{"rank": "?rank1"}, {"rank": "?rank1"}, {"rank": "?rank1"}], 
        "#seq-type": "allow-junk",
        "#name": "Three of a Kind", "#then": {"three-of-a-kind": "?rank1", "score": 4}}
engine.add_rule(rule)

# test it
hand = [
    {"rank": "J", "suit": "spades"}, 
    {"rank": "J", "suit": "hearts"},
    {"rank": "3", "suit": "clubs"},
    {"rank": "J", "suit": "diamonds"},
    {"rank": "7", "suit": "clubs"}]
    
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': 'j', 'score': 1},
 {'one-pair': 'j', 'score': 2},
 {'high-card': 'j', 'score': 1},
 {'high-card': '3', 'score': 1},
 {'one-pair': 'j', 'score': 2},
 {'three-of-a-kind': 'j', 'score': 4},
 {'one-pair': 'j', 'score': 2},
 {'high-card': 'j', 'score': 1},
 {'high-card': '7', 'score': 1}]


## Straight

With a straight, things get more interesting. We need to detect whenever we have 5 cards in a sequence, where each card is one rank higher than the card before it.

We can do this by first detecting the "mini-runs", where there is one card which is one higher than the card before it. Then we'll look for a number of mini-runs with are connected by the ending and beginning card.

To allow for this type of overlap in sequence, where the ending of one constituent can be the beginning of the next constituent, we set the #seq-type to "overlap-connected".

In [6]:
rules = [

    {"#when": [{"rank": "2"}, {"rank": "3"}],
     "#then": {"mini-run": "3"}},

    {"#when": [{"rank": "3"}, {"rank": "4"}],
        "#then": {"mini-run": "4"}},

    {"#when": [{"rank": "4"}, {"rank": "5"}],
        "#then": {"mini-run": "5"}},

    {"#when": [{"rank": "5"}, {"rank": "6"}],
        "#then": {"mini-run": "6"}},

    {"#when": [{"rank": "6"}, {"rank": "7"}],
        "#then": {"mini-run": "7"}},

    {"#when": [{"rank": "7"}, {"rank": "8"}],
        "#then": {"mini-run": "8"}},

    {"#when": [{"rank": "8"}, {"rank": "9"}],
        "#then": {"mini-run": "9"}},

    {"#when": [{"rank": "9"}, {"rank": "10"}],
        "#then": {"mini-run": "10"}},

    {"#when": [{"rank": "10"}, {"rank": "J"}],
        "#then": {"mini-run": "J"}},

    {"#when": [{"rank": "J"}, {"rank": "Q"}],
        "#then": {"mini-run": "Q"}},

    {"#when": [{"rank": "Q"}, {"rank": "K"}],
        "#then": {"mini-run": "K"}},

    {"#when": [{"rank": "K"}, {"rank": "A"}],
        "#then": {"mini-run": "A"}},

    {"#when": [{"mini-run": "?run1"}, {"mini-run": "?run2"}, {"mini-run": "?run3"}, {"mini-run": "?run4"}],
    "#seq-type": "overlap-connected",
        "#then": {"straight": "?run4", "score": 5}}
]
engine.add_rules(rules)

hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "4", "suit": "hearts"},
    {"rank": "5", "suit": "diamonds"},
    {"rank": "6", "suit": "clubs"},
    {"rank": "7", "suit": "spades"}]
    
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': '3', 'score': 1},
 {'mini-run': '4'},
 {'high-card': '4', 'score': 1},
 {'mini-run': '5'},
 {'high-card': '5', 'score': 1},
 {'mini-run': '6'},
 {'high-card': '6', 'score': 1},
 {'straight': '7', 'score': 5},
 {'high-card': '7', 'score': 1}]


## Flush

A flush is quite a bit easier. Similiar to the pair rule, we need to detect 5 cards in sequence which are share the same suit.

We do this by using the same ?suit1 variable in all five consituents in the pattern.

In [7]:
rule = {"#when": [{"suit": "?suit1"}, {"suit": "?suit1"}, {"suit": "?suit1"}, {"suit": "?suit1"}, {"suit": "?suit1"}],
     "#then": {"flush": "?suit1", "score": 6}}
engine.add_rule(rule)

hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "5", "suit": "spades"},
    {"rank": "7", "suit": "spades"},
    {"rank": "9", "suit": "spades"},
    {"rank": "J", "suit": "spades"}]
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': '3', 'score': 1},
 {'high-card': '5', 'score': 1},
 {'high-card': '7', 'score': 1},
 {'high-card': '9', 'score': 1},
 {'flush': 'spades', 'score': 6},
 {'high-card': 'j', 'score': 1}]


## Full House

A full house sounds trivial at first, but there's a catch. The cards can be arranged either with the three of a kind going first and the pair going second; the pair going first and the three of a kind going second; or the cards interlaced so that the pair is hiding in the middle of the three of a kind!

For this scenario, rather than look for a sequence of cards in order, we'll look for a "set", which means the cards can be arranged in any order, as long as the constituents do not contain any sub-constituents of each other. Fortunately the engine has an easy way to handle this, by setting the #seq-type to "set".

In [8]:
rule = {"#when": [{"three-of-a-kind": "?rank1"}, {"one-pair": "?rank2"}],
        "#seq-type": "set",
        "#then": {"full-house": {"three-of-a-kind": "?rank1", "one-pair": "?rank2"}, "score": 7}}
engine.add_rule(rule)

hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "7", "suit": "spades"},
    {"rank": "3", "suit": "hearts"},
    {"rank": "7", "suit": "hearts"},
    {"rank": "3", "suit": "diamonds"}]
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': '3', 'score': 1},
 {'high-card': '7', 'score': 1},
 {'one-pair': '3', 'score': 2},
 {'high-card': '3', 'score': 1},
 {'one-pair': '7', 'score': 2},
 {'high-card': '7', 'score': 1},
 {'one-pair': '3', 'score': 2},
 {'full-house': {'three-of-a-kind': '3', 'one-pair': '7'}, 'score': 7},
 {'one-pair': '3', 'score': 2},
 {'high-card': '3', 'score': 1}]


## Four of a Kind

A four of a kind returns to normalcy - just detect four cards that have the same rank, and allow for junk cards in between constituents.

Notice that the engine detects quite a bit of other patterns here too! That's OK, it is detecting legal sequences based on the previous rules. Just like in real poker, you can look at the cards in your hand in different ways, with the goal of picking the *highest* ranking arrangement in your hand.

In [9]:
rule = {"#when": [{"rank": "?rank1"}, {"rank": "?rank1"}, {"rank": "?rank1"}, {"rank": "?rank1"}],
        "#seq-type": "allow-junk",
        "#then": {"four-of-a-kind": "?rank1", "score": 8}}
engine.add_rule(rule)

hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "3", "suit": "clubs"},
    {"rank": "3", "suit": "hearts"},
    {"rank": "7", "suit": "hearts"},
    {"rank": "3", "suit": "diamonds"}]
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': '3', 'score': 1},
 {'one-pair': '3', 'score': 2},
 {'high-card': '3', 'score': 1},
 {'one-pair': '3', 'score': 2},
 {'three-of-a-kind': '3', 'score': 4},
 {'one-pair': '3', 'score': 2},
 {'high-card': '3', 'score': 1},
 {'high-card': '7', 'score': 1},
 {'one-pair': '3', 'score': 2},
 {'three-of-a-kind': '3', 'score': 4},
 {'one-pair': '3', 'score': 2},
 {'three-of-a-kind': '3', 'score': 4},
 {'four-of-a-kind': '3', 'score': 8},
 {'three-of-a-kind': '3', 'score': 4},
 {'two-pair': {'pair1': '3', 'pair2': '3'}, 'score': 3},
 {'high-card': '3', 'score': 1}]


## Straight Flush

Detecting straight flushes presents a challenge. We want to detect whenever the arrangement of cards matches both a straight *and* and flush, and the members in both sets can overlap.

At first it's tempting to use the #seq-type = "set" option above, which gets us part of the way there. However, by default this option does not allow the constituents to contain members that are already in other constituents in the pattern, and so would fail by itself.

We can relax this option, so that the matching *will* allow constituents to share members, by using the #seq-allow-multi option. In this way the straight can contain members from the flush, and vice-versa.

In [10]:
rule = {"#when": [{"straight": "?rank"}, {"flush": "?suit"}],
    "#seq-type": "set",
    "#seq-allow-multi": True,
     "#then": {"straight-flush": "?suit", "rank": "?rank", "score": 9}}
engine.add_rule(rule)

hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "4", "suit": "spades"},
    {"rank": "5", "suit": "spades"},
    {"rank": "6", "suit": "spades"},
    {"rank": "7", "suit": "spades"}]
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': '3', 'score': 1},
 {'mini-run': '4'},
 {'high-card': '4', 'score': 1},
 {'mini-run': '5'},
 {'high-card': '5', 'score': 1},
 {'mini-run': '6'},
 {'high-card': '6', 'score': 1},
 {'flush': 'spades', 'score': 6},
 {'straight-flush': 'spades', 'rank': '7', 'score': 9},
 {'high-card': '7', 'score': 1}]


## Royal Flush

Ah, the infamous royal flush. This turns out to be a special case of the straight flush rule, where the rank of the straight is an ace.

Easy enough:

In [11]:
rule = {"#when": [{"straight-flush": "?suit", "rank": "A"}],
        "#then": {"royal-flush": "?suit", "score": 10}}
engine.add_rule(rule)

hand = [
    {"rank": "10", "suit": "hearts"}, 
    {"rank": "J", "suit": "hearts"},
    {"rank": "Q", "suit": "hearts"},
    {"rank": "K", "suit": "hearts"},
    {"rank": "A", "suit": "hearts"}]
result = engine.process(hand, extract_conclusions=True)
pprint.pprint(result, sort_dicts=False)

[{'high-card': '10', 'score': 1},
 {'mini-run': 'J'},
 {'high-card': 'j', 'score': 1},
 {'mini-run': 'Q'},
 {'high-card': 'q', 'score': 1},
 {'mini-run': 'K'},
 {'high-card': 'k', 'score': 1},
 {'flush': 'hearts', 'score': 6},
 {'royal-flush': 'hearts', 'score': 10},
 {'high-card': 'a', 'score': 1}]


## Choosing the Highest Scoring Arrangement

Choosing the highest scoring arrangment is a matter of picking out the conclusion with the highest score.

Let's try it on a sample hand we used previously.

In [12]:
hand = [
    {"rank": "3", "suit": "spades"}, 
    {"rank": "4", "suit": "spades"},
    {"rank": "5", "suit": "spades"},
    {"rank": "6", "suit": "spades"},
    {"rank": "7", "suit": "spades"}]
result = engine.process(hand, extract_conclusions=True)
# pprint.pprint(result, sort_dicts=False)

max_score = max(filter(lambda x: "score" in x, result), key=lambda x: x['score'])
print("BEST RANKING:", max_score)

BEST RANKING: {'straight-flush': 'spades', 'rank': '7', 'score': 9}


## Conclusion

Detecting poker hands illustrates many of the concepts in sequence and set detection. Hopefully you found it as a fun way to learn how the engine can help with this kind of scenario.